In [9]:

pip install BeautifulSoup4==4.8.2


Note: you may need to restart the kernel to use updated packages.


In [10]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import os
import pymysql
import uuid
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import re
from datetime import datetime


In [11]:
juso_db = pymysql.connect(
    user='ssafy', 
    passwd='ssafy', 
    host='127.0.0.1', 
    db='ieum', 
    charset='utf8'
)



In [12]:
def getanimalnum(pageNum):
    ##다운 받을 주소
    baseUrl = 'https://www.zooseyo.or.kr/Yu_board/freesale_ty_1.html?animal=%EA%B0%95%EC%95%84%EC%A7%80&area=&ty=1&page='   
    url = baseUrl+str(pageNum)  
    req=requests.get(url)
    print(req)
    html = req.text
    soup = bs(html, "html.parser")
    info=soup.select('a[href]')
    checkpoint=False
    ct=0;
    animalnum=[]
    for i in info:
        tags=i.get('href')
        if("sell_up.html?gort=g"==tags):
            ct+=1
        num=tags.split("&")[0].split("?")
        if(ct>1 and num[0]=="view.html"):
            animalnum.append(num[1].split("=")[1])
            
    return animalnum

In [13]:
def getinfo(animalNum):
    preUrl='https://www.zooseyo.or.kr/Yu_board/view.html?no='
    postUrl='&page=1&animal=&area='
    url=preUrl+animalNum+postUrl
    dogsinfo=[]
    
    req=requests.get(url)
    html = req.text
    soup = bs(html, "html.parser")

    #개번호 등록
    dogsinfo.append(animalNum)    
    
    #url 등록
    dogsinfo.append(url)

    
    ##img 경로등록
    imginfo=soup.select('img[src]')
    imglist=[]
    for i in imginfo:
        tags=i.get('src')
        if('/dog_sale/' in tags):
#             print(tags)
            imglist.append('https://www.zooseyo.or.kr'+tags)
    dogsinfo.append(imglist)

    ##개정보 등록
    infos=soup.get_text().split()
    
   ##날짜 인덱스 찾기
    #print(infos)
    for i in range(len(infos)):
        try:
            if"무료분양" in infos[i]:
                dogsinfo.append(infos[i].split("(")[1])
                break
        except:
            dogsinfo.append("")
    
    
    
#     idx=0
#     for i in range(300,500):
#         print(i,infos[i] )
        
    
    for idx in [356,364,366,368]:
        dogsinfo.append(infos[idx])
    

    
    
    return dogsinfo

In [14]:
def saveimg(directory,url):    
    urls=url.split("/")
    extension=urls[len(urls)-1].split(".")[1]
    date=datetime.today()
    month=int(datetime.today().month/10)*10+int(datetime.today().month%10)
    day=int(datetime.today().day/10)*10+int(datetime.today().day%10)
    path1=str(datetime.today().year)+str(month)+ str(day)
    
    n=1
    imguuid=uuid.uuid1();
    save_path= directory+"/"+path1
    save_root = directory+"/"+path1
    if not os.path.exists(save_root):os.makedirs(save_root)
    try:
        with urlopen(url) as f:
            with open(os.path.join(save_path, str(imguuid)+"."+extension),'wb') as h: # w - write b - binary
                img = f.read()
                h.write(img)
        print(f"directory :  {save_path}  파일명 : {imguuid} 원본 파일명 : {urls[len(urls)-1]}")
    except:
        print("error")
    return [save_path,str(imguuid)+"."+extension,"",extension,urls[len(urls)-1]]
    

In [15]:
def saveAndInsert(dogInfos):
    dognum=info[0]
    detailurl=info[1]
    imgurls=info[2]
    date=info[3].split(".")
    date=date[0]+"-"+date[1]+"-"+date[2]
    breed=info[4]
    location=info[5]
    sex=info[6]
    phoneNumRegex = re.compile(r'\d{3}-\d{4}-\d{4}')
    phone=phoneNumRegex.findall(info[7])
    if phone==[]:
        phone=""
    else:
        phone=phone[0]
        
    ismixed=False
    
    cursor = juso_db.cursor(pymysql.cursors.DictCursor)
    
    #######################파일 저장############################
    #server
    #dir="/home/upload/img"
    #local    
    dir="C://Users//sskim//Downloads"
    orginnamelist=[]
    systemimgpaths=[]
    predictlst=[]
    try:        
        for i in range(1,len(imgurls)):
            saveresult=saveimg(dir,imgurls[i])

            sql="INSERT INTO `ieum`.`file`(`path`,`system_name`,`size`,`type`,`origin_name`) "+\
            f"VALUES('{saveresult[0]}','{saveresult[1]}','0','{saveresult[3]}','{saveresult[4]}'); "

            cursor.execute(sql)
            result = cursor.fetchall()
            juso_db.commit()        
            orginnamelist.append(saveresult[4])
            systemimgpaths.append(saveresult[0]+saveresult[1])
        print("file insert  done")
        print(orginnamelist)
        print(systemimgpaths)
    except Exception  as e:
        print("file insert error",e)
        
    
    #####################insert doginfo table##########################
    if "믹스" in breed:
        ismixed=True
        #ex [(predict_breed,percent)]
        #predictlst=fun(systemimgpaths)
        maxpercent=0;
        for predict in predictlst:
            if predict[1]>maxpercent:
                breed=predict[0]
                maxpercent=predict[1]
        
        
    #####################insert doginfo table##########################
    try:

        sql = "INSERT INTO `ieum`.`doginfo`(`id`,`breed`,`location`,`url`,`phone`,`datetime`,`sex`)"+\
        f" VALUES('{dognum}','{breed}','{location}','{detailurl}','{phone}','{date}','{sex}'); "

        cursor.execute(sql)
        result = cursor.fetchall()
        juso_db.commit()        
        print("doginfo insert done")
    except Exception  as e:
        print("doginfo insert error",e)
        return 

    #####################insert doginfoimages table##########################
    try:
        for orginname in orginnamelist:
            sql = f"SELECT id FROM `ieum`.`file` WHERE origin_name='{orginname}' ;"
            cursor.execute(sql)
            fileId = cursor.fetchall()
            print("fileId ",fileId[0]['id'])
            sql="INSERT INTO `ieum`.`doginfoimage`(`dogid`,`file_id`)" + \
            f" VALUES('{dognum}','{fileId[0]['id']}'); "
            cursor.execute(sql)
            fileId = cursor.fetchall()
        print("doginfoimages insert  done")
    except Exception  as e:
        print("doginfoimages insert  done",e)
        
    ####################insert doginfopredict table##########################
    
    if ismixed==False :
        return 
    
    try:
        for predict in predictlst:
            sql="INSERT INTO `ieum`.`doginfopredict`(`dogid`,`percent`,`predicted_breed`) "+ \
            f"VALUES('{dognum}','{predict[1]}','{predict[0]}'); "
            cursor.execute(sql)
            fileId = cursor.fetchall()
        print("doginfopredict insert  done")
    except Exception  as e:
        print("doginfopredict insert  done",e)
    
    
    


In [16]:
date=datetime.today()
month=int(datetime.today().month/10)*10+int(datetime.today().month%10)
day=int(datetime.today().day/10)*10+int(datetime.today().day%10)
endDate=str(datetime.today().year)+str(month)+ str(day)
    

In [19]:
if __name__ == "__main__":
    
    #크롤링 멈추는 날짜(등록일 기준)
    date=datetime.today()
    month=str(int(datetime.today().month/10)*10)+str(datetime.today().month%10)
    day=int(datetime.today().day/10)*10+int(datetime.today().day%10)
    endDate=str(datetime.today().year)+"."+str(month) +"."+str(day)
    print("crawling :",endDate)
    ##if you want 
    
    dogInfos=[]
    ##넘겨볼 페이지 갯수
    for pageNum in range(0,4):
        animalNums=getanimalnum(pageNum)
        cnt=0
        for num in animalNums:
            cnt+=1
            
            info=getinfo(num)
            if(endDate not in info[len(info)-5] ):
                break
            print("====개정보===")
            print("pageNum : ",pageNum,info)
            print(info[len(info)-5])            
            dogInfos.append(info)
    print("####"*20)
    print("####"*20)
    print(f'Data Cnt: {len(dogInfos)}')
    print("####"*20)
    print("####"*20)
    
    
    
    
        
    

crawling : 2021.04.1
<Response [200]>
====개정보===
pageNum :  0 ['338543', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338543&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617234997_77236300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617234997_77236300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617234998_62636600.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617234999_29505800.jpg'], '2021.04.01', '믹스', '경기', '남아', '보더콜리']
2021.04.01
====개정보===
pageNum :  0 ['338539', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338539&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617218288_53978400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617218288_53978400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617218289_74509300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202104/1617218289_76183100.jpg', 'https://www.zooseyo.or.kr/dog_sale/phot

====개정보===
pageNum :  0 ['338455', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338455&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617108044_84025200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617108044_84025200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617108049_06617400.jpg'], '2021.03.30', '이탈리안그레이하운드', '서울', '남아', '분양보내요..']
2021.03.30
====개정보===
pageNum :  0 ['338451', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338451&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617106219_92232800.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617106219_92232800.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617106220_02925200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617106220_05087400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617106220_07860700.jpg'], '2021.03.30', '시츄', '서울', '남아', '/']
2021.03.30
====개정보===
pageNum

====개정보===
pageNum :  1 ['338470', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338470&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617131888_82609900.JPG', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617131888_82609900.JPG', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617131889_03508100.JPG', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617131889_05331500.JPG', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617131889_06986300.JPG'], '2021.03.31', '진도믹스', '인천', '2021-03-01', '010-4968-2558']
2021.03.31
====개정보===
pageNum :  1 ['338468', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338468&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617125485_25366100.png', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1617125485_25366100.png'], '2021.03.31', '시바믹스', '전국분양', '여아', '010-2883-1725']
2021.03.31
====개정보===
pageNum :  1 ['338460', 'https://www.zooseyo.or.kr/Yu_board/vie

====개정보===
pageNum :  2 ['338374', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338374&page=1&animal=&area=', [], '2021.03.29', '포메라니안', '광주', '남아', '02--']
2021.03.29
====개정보===
pageNum :  2 ['338342', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338342&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616999925_14111900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616999925_14111900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616999925_48737700.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616999925_60831400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616999925_70254100.jpg'], '2021.03.29', '-', '진돗개', '시바', '오현석']
2021.03.29
====개정보===
pageNum :  2 ['338335', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338335&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616992378_90733900.jpeg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616992378_9

====개정보===
pageNum :  3 ['338316', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338316&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988486_35835300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988486_35835300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988486_50602400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988486_53031800.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988486_54322800.jpg'], '2021.03.29', '믹스견', '전국분양', '남아', '선택--']
2021.03.29
====개정보===
pageNum :  3 ['338315', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=338315&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988374_52149500.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988374_52149500.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988374_68607600.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616988374_71751900.j

In [ ]:
##
for info in dogInfos:
    saveAndInsert(info)




directory :  C://Users//sskim//Downloads/2021331  파일명 : 3f92596e-91f3-11eb-8a49-08d23e250ad9 원본 파일명 : 1617173272_68088600.jpeg
directory :  C://Users//sskim//Downloads/2021331  파일명 : 3fcaf576-91f3-11eb-a109-08d23e250ad9 원본 파일명 : 1617173272_78543000.jpeg
directory :  C://Users//sskim//Downloads/2021331  파일명 : 400cb910-91f3-11eb-8f91-08d23e250ad9 원본 파일명 : 1617173272_80365100.jpeg
directory :  C://Users//sskim//Downloads/2021331  파일명 : 403a31d4-91f3-11eb-ae12-08d23e250ad9 원본 파일명 : 1617173272_82277100.jpeg
file insert  done
['1617173272_68088600.jpeg', '1617173272_78543000.jpeg', '1617173272_80365100.jpeg', '1617173272_82277100.jpeg']
['C://Users//sskim//Downloads/20213313f92596e-91f3-11eb-8a49-08d23e250ad9.jpeg', 'C://Users//sskim//Downloads/20213313fcaf576-91f3-11eb-a109-08d23e250ad9.jpeg', 'C://Users//sskim//Downloads/2021331400cb910-91f3-11eb-8f91-08d23e250ad9.jpeg', 'C://Users//sskim//Downloads/2021331403a31d4-91f3-11eb-ae12-08d23e250ad9.jpeg']
doginfo insert error (1062, "Duplicate e

In [53]:
"""
for info in dogInfos:
    if(info[2] == []) :
        continue
    for img in info[2][1:]:
        continue
        ##요기에 원하는 경로
##        saveimg("C:/Users/sskim/Downloads/Dogs",img,info[len(info)-3],info[0])
"""
    
            

'\nfor info in dogInfos:\n    if(info[2] == []) :\n        continue\n    for img in info[2][1:]:\n        continue\n        ##요기에 원하는 경로\n##        saveimg("C:/Users/sskim/Downloads/Dogs",img,info[len(info)-3],info[0])\n'